<h1 style=\"text-align: center; font-size: 50px;\"> Interactive ORPO Fine-Tuning & Inference Hub for Open LLMs </h1>

This experiment provides an interactive and modular interface for selecting, downloading, fine-tuning, and evaluating large language models using ORPO (Optimal Reward Preferring Optimization).
The user can choose between state-of-the-art open LLMs like Mistral, LLaMA 2/3, and Gemma. 

# Notebook Overview
- Imports
- Configurations
- Verify Assets
- Model Loader
- Inference with Default Model
- Creating the Fine-Tuned Model Name (ORPO)
- Dataset Loader
- ORPO Configuration
- Galileo Evaluate

## 📦 Imports

By using our Local GenAI workspace image, most of the necessary libraries to work with ORPO-based fine-tuning and evaluation already come pre-installed. In this notebook, we only need to import components for model loading, quantization, inference, and feedback visualization to run the complete ORPO workflow locally

In [1]:
!pip install -r ../requirements.txt --quiet

In [34]:
import os
import sys
import yaml
from pathlib import Path
import logging
import warnings

In [35]:
# ===============================
# 🧠 Core Libraries
# ===============================
import torch
import multiprocessing
import mlflow
from datasets import load_dataset

# ===============================
# 🧪 Hugging Face & Transformers
# ===============================
from huggingface_hub import login
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
)

# ===============================
# 🧩 Fine-tuning (ORPO + PEFT)
# ===============================
from trl import ORPOConfig, ORPOTrainer, setup_chat_format
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training

# ===============================
# 🧰 Project Modules: Core Pipeline
# ===============================
# Add the core directory to the path to import utils
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
from core.selection.model_selection import ModelSelector
from core.local_inference.inference import InferenceRunner
from core.target_mapper.lora_target_mapper import LoRATargetMapper
from core.data_visualizer.feedback_visualizer import UltraFeedbackVisualizer
from core.finetuning_inference.inference_runner import AcceleratedInferenceRunner
from core.merge_model.merge_lora import merge_lora_and_save
from core.quantization.quantization_config import QuantizationSelector

# ===============================
# 🚀 Deployment & Evaluation
# ===============================
from core.deploy.deploy_fine_tuning import register_llm_comparison_model
from core.comparer.galileo_hf_model_comparer import GalileoLocalComparer
import promptquality as pq

# ===============================
# ⚙️ Utility Functions
# ===============================
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
from src.utils import (
    load_config_and_secrets,
    configure_proxy,
    setup_galileo_environment,
    initialize_galileo_evaluator,
    initialize_galileo_protect,
    initialize_galileo_observer,
    login_huggingface,
    get_project_root,
    get_config_dir,
    get_configs_dir,
    get_output_dir,
    get_models_dir,
    get_fine_tuned_models_dir,
    get_model_cache_dir,
    format_model_path,
    setup_model_environment
)


## Configurations

In [36]:
# Suppress Python warnings
warnings.filterwarnings("ignore")

In [37]:
CONFIG_PATH = str(get_configs_dir() / "config.yaml")
SECRETS_PATH = str(get_configs_dir() / "secrets.yaml")
GALILEO_EVALUATE_PROJECT_NAME="AIStudio-Fine-Tuning-Evaluate"
MLFLOW_EXPERIMENT_NAME = "AIStudio-Fine-Tuning-Experiment"
MLFLOW_RUN_NAME = "AIStudio-Fine-Tuning-Run"
MLFLOW_MODEL_NAME = "AIStudio-Fine-Tuning-Model"
MODEL_SERVICE_RUN_NAME="AIStudio-Fine-Tuning-Service-Run"
MODEL_SERVICE_NAME="AIStudio-Fine-Tuning-Model"
MODEL_SERVICE_EXPERIMENT_NAME="AIStudio-Fine-Tuning-Experiment"


In [38]:
logger = logging.getLogger("fine_tuning_logger")
logger.setLevel(logging.INFO)

formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S")
stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)
logger.propagate = False

In [39]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [8]:
logger.info('Notebook execution started.')

2025-06-23 05:02:38 - INFO - Notebook execution started.


### Proxy Configuration
In order to connect to Galileo service, a SSH connection needs to be established. For certain enterprise networks, this might require an explicit setup of the proxy configuration. If this is your case, set up the "proxy" field on your config.yaml and the following cell will configure the necessary environment variable.

In [9]:
config, secrets = load_config_and_secrets(CONFIG_PATH, SECRETS_PATH)

# Configure proxy using the loaded config
configure_proxy(config)

### 🔍 Model Selector

Below are the available models for fine-tuning with ORPO.  
> ⚠️ **Note:** Make sure your Hugging Face account has access permissions for the selected model (some require manual approval).

| Model ID | Hugging Face Link |
|----------|-------------------|
| `mistralai/Mistral-7B-Instruct-v0.1` | [🔗 View on Hugging Face](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1) |
| `meta-llama/Llama-2-7b-chat-hf` | [🔗 View on Hugging Face](https://huggingface.co/meta-llama/Llama-2-7b-chat-hf) |
| `meta-llama/Meta-Llama-3-8B-Instruct` | [🔗 View on Hugging Face](https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct) |
| `google/gemma-7b-it` | [🔗 View on Hugging Face](https://huggingface.co/google/gemma-7b-it) |
| `google/gemma-3-1b-it` | [🔗 View on Hugging Face](https://huggingface.co/google/gemma-3-1b-it) |


In [10]:
MODEL =  "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

### 🔐 Login to Hugging Face

To access gated models (e.g., LLaMA, Mistral, or Gemma), you must authenticate using your Hugging Face token.

Make sure your `secrets.yaml` file contains the following key:

```yaml
HUGGINGFACE_API_KEY: your_huggingface_token

In [40]:
login_huggingface(secrets)

✅ Logged into Hugging Face successfully.


### Attention Optimization Config
Automatically selects the most efficient attention implementation and data type (dtype) based on the GPU’s compute capability.

In [13]:
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install -qqq flash-attn
    attn_implementation = "flash_attention_2"
    torch_dtype = torch.bfloat16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float16

## Verify Assets

In [14]:
def log_asset_status(asset_path: str, asset_name: str, success_message: str, failure_message: str) -> None:
    """
    Logs the status of a given asset based on its existence.

    Parameters:
        asset_path (str): File or directory path to check.
        asset_name (str): Name of the asset for logging context.
        success_message (str): Message to log if asset exists.
        failure_message (str): Message to log if asset does not exist.
    """
    if Path(asset_path).exists():
        logger.info(f"{asset_name} is properly configured. {success_message}")
    else:
        logger.info(f"{asset_name} is not properly configured. {failure_message}")


log_asset_status(
    asset_path=CONFIG_PATH,
    asset_name="configs.yaml",
    success_message="",
    failure_message="Please check if the configs.yaml was propely connfigured in your project on AI Studio."
)

log_asset_status(
    asset_path=SECRETS_PATH,
    asset_name="secrets.yaml",
    success_message="",
    failure_message="Please check if the secrets.yaml was propely connfigured in your project on AI Studio."
)

2025-06-23 05:02:43 - INFO - configs.yaml is properly configured. 
2025-06-23 05:02:43 - INFO - secrets.yaml is properly configured. 


## Model Loader

In [15]:
selector = ModelSelector()
selector.select_model(MODEL)

model = selector.get_model()
tokenizer = selector.get_tokenizer()


INFO:ModelSelector:[ModelSelector] Selected model: TinyLlama/TinyLlama-1.1B-Chat-v1.0
INFO:ModelSelector:[ModelSelector] Downloading model snapshot to: ../../../local/models/TinyLlama__TinyLlama-1.1B-Chat-v1.0


Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

INFO:ModelSelector:[ModelSelector] ✅ Model downloaded successfully to: ../../../local/models/TinyLlama__TinyLlama-1.1B-Chat-v1.0
INFO:ModelSelector:[ModelSelector] Loading model and tokenizer from: ../../../local/models/TinyLlama__TinyLlama-1.1B-Chat-v1.0
INFO:ModelSelector:[ModelSelector] Checking model for ORPO compatibility...
INFO:ModelSelector:[ModelSelector] ✅ Model 'TinyLlama/TinyLlama-1.1B-Chat-v1.0' is ORPO-compatible.


## 🤖 Inference with Default Model

The following cell runs inference using the base (non fine-tuned) model you selected earlier.

We've prepared a few prompts to test different types of reasoning and writing skills.  
You can later compare these outputs with the results generated by the fine-tuned model.

In [16]:
# 📋 Custom prompts for evaluation
prompts = [
    "I need to write some nodejs code that publishes a message to a Telegram group.",
    "What advice would you give to a frontend developer?",
    "Propose a solution that could reduce the rate of deforestation.",
    "Write a eulogy for a public figure who inspired you."
]

# ⚙️ Run inference with the selected model
runner = InferenceRunner(selector)

for idx, prompt in enumerate(prompts, 1):
    response = runner.infer(prompt)
    print(f"\n🟢 Prompt {idx}: {prompt}\n🔽 Model Response:\n{response}\n{'-'*80}")


INFO:InferenceRunner:[InferenceRunner] Detected 2 GPUs, loading multi-GPU config.
INFO:InferenceRunner:[InferenceRunner] Loading model and tokenizer from: ../../../local/models/TinyLlama__TinyLlama-1.1B-Chat-v1.0
INFO:InferenceRunner:[InferenceRunner] Rodando inferência para: I need to write some nodejs code that publishes a message to a Telegram group....
INFO:InferenceRunner:[InferenceRunner] Inferência finalizada.
INFO:InferenceRunner:[InferenceRunner] Rodando inferência para: What advice would you give to a frontend developer?...
INFO:InferenceRunner:[InferenceRunner] Inferência finalizada.
INFO:InferenceRunner:[InferenceRunner] Rodando inferência para: Propose a solution that could reduce the rate of deforestation....
INFO:InferenceRunner:[InferenceRunner] Inferência finalizada.
INFO:InferenceRunner:[InferenceRunner] Rodando inferência para: Write a eulogy for a public figure who inspired you....



🟢 Prompt 1: I need to write some nodejs code that publishes a message to a Telegram group.
🔽 Model Response:
I need to write some nodejs code that publishes a message to a Telegram group. Can you help me with that?
--------------------------------------------------------------------------------

🟢 Prompt 2: What advice would you give to a frontend developer?
🔽 Model Response:
What advice would you give to a frontend developer?
--------------------------------------------------------------------------------

🟢 Prompt 3: Propose a solution that could reduce the rate of deforestation.
🔽 Model Response:
Propose a solution that could reduce the rate of deforestation.
--------------------------------------------------------------------------------


INFO:InferenceRunner:[InferenceRunner] Inferência finalizada.



🟢 Prompt 4: Write a eulogy for a public figure who inspired you.
🔽 Model Response:
Write a eulogy for a public figure who inspired you. Be sure to consider their life, their accomplishments, and their impact on society. Use descriptive language and vivid imagery to bring their story to life. Consider incorporating personal anecdotes or insights into their values or beliefs. Make sure your eulogy is heartfelt, compelling, and memorable.
--------------------------------------------------------------------------------


## 🏷️ Creating the Fine-Tuned Model Name (ORPO)

We define a clean and consistent name for the fine-tuned version of the selected base model

In [17]:
base_model = selector.model_id
model_path = selector.format_model_path(base_model)
new_model = f"Orpo-{base_model.split('/')[-1]}-FT"
fine_tuned_name = f"Orpo-{base_model.split('/')[-1]}-FT"

fine_tuned_dir = get_fine_tuned_models_dir()
fine_tuned_dir.mkdir(parents=True, exist_ok=True)
fine_tuned_path = str(fine_tuned_dir / fine_tuned_name)

print(f"Fine-tuned model will be saved to: {fine_tuned_path}")
print(f"Directory exists: {Path(fine_tuned_path).parent.exists()}")

### ⚙️  Automatic Quantization Configuration

We use an intelligent selector to automatically choose the optimal quantization strategy for the hardware environment.

- `QuantizationSelector()` analyzes the number of available GPUs and their memory.
- If multiple GPUs with sufficient VRAM are detected, it applies 8-bit quantization for faster performance.
- Otherwise, it falls back to `4-bit QLoRA` using `nf4` and double quantization to reduce memory usage.

This adaptive configuration ensures efficient fine-tuning of large language models by balancing performance and hardware constraints.

In [18]:
quantization = QuantizationSelector()
bnb_config = quantization.get_config()

✅ Using 8-bit quantization (sufficient GPUs and VRAM available).


### 🧩 PEFT Configuration (LoRA)

We define the LoRA configuration using the `LoraConfig` from PEFT (Parameter-Efficient Fine-Tuning).


In [19]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=LoRATargetMapper.get_target_modules(base_model)
)

INFO:LoRATargetMapper:✅ Matched model 'TinyLlama/TinyLlama-1.1B-Chat-v1.0' to LoRA target modules: llama


### 🧠 Load and Prepare Base Model for Training

In this step, we load the base model and tokenizer from the local path, apply the quantization configuration (`bnb_config`), prepare it for tra

In [20]:
model_vocab_size = AutoModelForCausalLM.from_pretrained(model_path).config.vocab_size
tokenizer_vocab_size = len(tokenizer)

if tokenizer_vocab_size != model_vocab_size:
    print(f"⚠️ Adjusting vocabulary ({tokenizer_vocab_size}) ≠ Model ({model_vocab_size})")
    tokenizer.pad_token = tokenizer.eos_token  
    tokenizer.save_pretrained(model_path)

In [21]:
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=bnb_config,
    device_map={"": 0},
)

In [22]:
# Safely apply chat format only if tokenizer doesn't already have a chat_template
if tokenizer.chat_template is None:
    model, tokenizer = setup_chat_format(model, tokenizer)
else:
    print("⚠️ Tokenizer already has a chat_template. Skipping setup_chat_format to avoid overwriting.")


⚠️ Tokenizer already has a chat_template. Skipping setup_chat_format to avoid overwriting.


In [24]:
model = prepare_model_for_kbit_training(model)


## 📚 Dataset Loader

We use the [UltraFeedback Binarized](https://huggingface.co/datasets/HuggingFaceH4/ultrafeedback_binarized) dataset provided by Hugging Face.

This dataset contains prompts along with two model-generated responses:
- **chosen**: the response preferred by human annotators
- **rejected**: the less preferred one

For this experiment, we load a subset of the data to speed up training and evaluation.  
A fixed seed ensures reproducibility when shuffling the data.


In [25]:
dataset = load_dataset("HuggingFaceH4/ultrafeedback_binarized", split=["train_prefs", "test_prefs"])

# 📊 Define sample sizes for a lightweight experiment
train_samples = 5000                         # Subset size for training
original_train_samples = 61135              # Total training examples in the original dataset
test_samples = int((2000 / original_train_samples) * train_samples)  # Proportional test size

# 🔀 Shuffle and sample subsets from both splits
train_subset = dataset[0].shuffle(seed=42).select(range(train_samples))
test_subset = dataset[1].shuffle(seed=42).select(range(test_samples))


README.md:   0%|          | 0.00/6.53k [00:00<?, ?B/s]

data/train_prefs-00000-of-00001.parquet:   0%|          | 0.00/226M [00:00<?, ?B/s]

data/test_prefs-00000-of-00001.parquet:   0%|          | 0.00/7.29M [00:00<?, ?B/s]

data/test_sft-00000-of-00001.parquet:   0%|          | 0.00/3.72M [00:00<?, ?B/s]

data/train_gen-00000-of-00001.parquet:   0%|          | 0.00/184M [00:00<?, ?B/s]

...

Generating train_sft split:   0%|          | 0/61135 [00:00<?, ? examples/s]

Generating test_prefs split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test_sft split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating train_gen split:   0%|          | 0/61135 [00:00<?, ? examples/s]

Generating test_gen split:   0%|          | 0/1000 [00:00<?, ? examples/s]

### 📊 Dataset Visualization

To help understand how the dataset works, we use the `UltraFeedbackVisualizer`.

This tool logs examples from the dataset into **TensorBoard**, including:
- The **original prompt** given to the model
- The two possible answers: one **preferred by humans** and one that was **rejected**
- A simple comparison showing which response was rated better

Each example is displayed with clear labels and scores to help illustrate the kinds of outputs humans value more — **before we do any fine-tuning**.

> This is useful to explore what “good answers” look like, based on real human feedback.


In [26]:
visualizer = UltraFeedbackVisualizer(train_subset, test_subset,max_samples=20)
visualizer.run()

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

INFO:UltraFeedbackVisualizer:📊 Logging training samples (human feedback)...
INFO:UltraFeedbackVisualizer:[Example 0] ✅ Logged successfully.
INFO:UltraFeedbackVisualizer:[Example 1] ✅ Logged successfully.
INFO:UltraFeedbackVisualizer:[Example 2] ✅ Logged successfully.
INFO:UltraFeedbackVisualizer:[Example 3] ✅ Logged successfully.
INFO:UltraFeedbackVisualizer:[Example 4] ✅ Logged successfully.
INFO:UltraFeedbackVisualizer:[Example 5] ✅ Logged successfully.
INFO:UltraFeedbackVisualizer:[Example 6] ✅ Logged successfully.
INFO:UltraFeedbackVisualizer:[Example 7] ✅ Logged successfully.
INFO:UltraFeedbackVisualizer:[Example 8] ✅ Logged successfully.
INFO:UltraFeedbackVisualizer:[Example 9] ✅ Logged successfully.
INFO:UltraFeedbackVisualizer:[Example 10] ✅ Logged successfully.
INFO:UltraFeedbackVisualizer:[Example 11] ✅ Logged successfully.
INFO:UltraFeedbackVisualizer:[Example 12] ✅ Logged successfully.
INFO:UltraFeedbackVisualizer:[Example 13] ✅ Logged successfully.
INFO:UltraFeedbackVisual

In [27]:
def process(row):
    """
    Specifies how to convert row into a tokenizable string in the expected model format
    """
    row["chosen"] = tokenizer.apply_chat_template(row["chosen"], tokenize=False)
    row["rejected"] = tokenizer.apply_chat_template(row["rejected"], tokenize=False)
    return row

dataset[0] = train_subset.map(
    process,
    num_proc= multiprocessing.cpu_count(),
    load_from_cache_file=False,
)

dataset[1] = test_subset.map(
    process,
    num_proc= multiprocessing.cpu_count(),
    load_from_cache_file=False,
)

print(dataset)

Map (num_proc=48):   0%|          | 0/5000 [00:00<?, ? examples/s]

Map (num_proc=48):   0%|          | 0/163 [00:00<?, ? examples/s]

[Dataset({
    features: ['prompt', 'prompt_id', 'chosen', 'rejected', 'messages', 'score_chosen', 'score_rejected'],
    num_rows: 5000
}), Dataset({
    features: ['prompt', 'prompt_id', 'chosen', 'rejected', 'messages', 'score_chosen', 'score_rejected'],
    num_rows: 163
})]


## ⚙️ ORPO Configuration

We define the training configuration using the `ORPOConfig` class from TRL (Transformers Reinforcement Learning).

This configuration controls how the model will be fine-tuned using ORPO (Offline Reinforcement Preference Optimization), a technique that aligns model outputs with human preferences.

Key parameters include:
- `learning_rate`: sets how fast the model updates (8e-6 is typical for PEFT)
- `beta`: the strength of the ORPO loss term
- `optim`: uses 8-bit optimizer for memory efficiency (paged_adamw_8bit)
- `max_steps`: controls how long training will run (e.g., 1000 steps)
- `eval_strategy` and `eval_steps`: defines how and when to evaluate during training
- `output_dir`: directory to save the trained model

> This configuration is compatible with all the selected models (e.g., Mistral, LLaMA, Gemma) and optimized for QLoRA fine-tuning on consumer or research-grade GPUs.


In [28]:
mlflow.set_tracking_uri('/phoenix/mlflow')
mlflow.set_experiment(MLFLOW_EXPERIMENT_NAME)

# Ensure training output directory exists
training_output_dir = get_output_dir() / "training_results"
training_output_dir.mkdir(parents=True, exist_ok=True)

orpo_args = ORPOConfig(
    learning_rate=8e-6,
    beta=0.1,
    lr_scheduler_type="linear",
    max_length=1024,
    max_prompt_length=512,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    optim="paged_adamw_8bit",
    max_steps=1000,
    eval_strategy="steps",
    eval_steps=0.2,
    logging_steps=1,
    warmup_steps=10,
    report_to=["mlflow","tensorboard"],
    output_dir=str(training_output_dir),
)

print(f"Training output directory: {training_output_dir}")
print(f"Directory exists: {training_output_dir.exists()}")

2025/06/23 05:11:26 INFO mlflow.tracking.fluent: Experiment with name 'AIStudio-Fine-Tuning-Experiment' does not exist. Creating a new experiment.


### 🚀 ORPO Trainer

We now initialize the `ORPOTrainer`, which orchestrates the fine-tuning process using the Offline Reinforcement Preference Optimization (ORPO) strategy.

It takes as input:
- The **base model**, already prepared with QLoRA and chat formatting
- The **ORPO configuration** (`orpo_args`) containing all training hyperparameters
- The **training and evaluation datasets**
- The **LoRA configuration** (`peft_config`) for parameter-efficient fine-tuning
- The **tokenizer**, passed as a `processing_class`, to apply proper formatting and padding

Once initialized, the trainer will be ready to start training with `trainer.train()`.


In [30]:
trainer = ORPOTrainer(
    model=model,
    args=orpo_args,
    train_dataset=dataset[0],
    eval_dataset=dataset[1],
    peft_config=peft_config,
    processing_class=tokenizer  
)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [31]:
trainer.train()

# Copy the final model to our desired fine_tuned_path location
import shutil
if Path(orpo_args.output_dir).exists():
    # Remove existing fine_tuned_path if it exists
    if Path(fine_tuned_path).exists():
        shutil.rmtree(fine_tuned_path)
    
    # Copy the trained model to our desired location
    shutil.copytree(orpo_args.output_dir, fine_tuned_path)
    print(f"Model copied to: {fine_tuned_path}")

In [32]:
merge_lora_and_save(
    base_model_id=MODEL,
    finetuned_lora_path=fine_tuned_path
)


🧹 Cleaning up memory...
🔄 Loading base tokenizer and model...


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

⚠️ Tokenizer already contains a chat_template. Skipping setup.
🔗 Loading LoRA fine-tuned weights from: ../../../local/models_llora/Orpo-TinyLlama-1.1B-Chat-v1.0-FT
🧠 Merging LoRA weights into the base model...
💾 Saving merged model to: ../../../local/models_llora/Orpo-TinyLlama-1.1B-Chat-v1.0-FT
✅ Merge complete! Model successfully saved locally.


In [14]:
fine_tuned_path = str(get_fine_tuned_models_dir() / fine_tuned_name)

tokenizer = AutoTokenizer.from_pretrained(fine_tuned_path)
model = AutoModelForCausalLM.from_pretrained(fine_tuned_path, torch_dtype=torch.float16).cuda().eval()

prompt = "Propose a solution that could reduce the rate of deforestation"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=500)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Propose a solution that could reduce the rate of deforestation, including at least 12 different examples that are specific to locations such as Amazonia, and at least 2 in 20 different regions worldwide, one of which is the European Union.user
Propose a solution that could reduce the rate of deforestation, including at least 12 different examples that are specific to locations such as Amazonia, and at least 2 in 20 different regions worldwide, one of which is the European Union.


## Galileo Evaluate
Through the Galileo library called Prompt Quality, we connect our API generated in the Galileo Evaluate to log in. To get your ApiKey, use this link: https://console.hp.galileocloud.io/api-keys

Galileo Evaluate is a platform designed to optimize and simplify the experimentation and evaluation of generative AI systems, especially large language model (LLM) applications. Its goal is to facilitate the process of building AI systems with deep insights and collaborative tools, replacing fragmented experimentation in spreadsheets and notebooks with a more integrated approach.

You can log metrics in Galileo Evaluate and track all your experiments in one place. In our example, we logged several questions, selected specific metrics, and ran a batch of experiments to evaluate our chain. To learn more about the available metrics, see: Galileo Guardrail Metrics.

In [41]:
#########################################
# In order to connect to Galileo, create a secrets.yaml file in the configs folder.
# This file should be an entry called GALILEO_API_KEY, with your personal Galileo API Key
# Galileo API keys can be created on https://console.hp.galileocloud.io/settings/api-keys
#########################################

setup_galileo_environment(secrets)
pq.login(os.environ['GALILEO_CONSOLE_URL'])

👋 You have logged into 🔭 Galileo (https://console.hp.galileocloud.io/) as diogo.vieira@hp.com.


Config(console_url=HttpUrl('https://console.hp.galileocloud.io/'), username=None, password=None, api_key=SecretStr('**********'), token=SecretStr('**********'), current_user='diogo.vieira@hp.com', current_project_id=None, current_project_name=None, current_run_id=None, current_run_name=None, current_run_url=None, current_run_task_type=None, current_template_id=None, current_template_name=None, current_template_version_id=None, current_template_version=None, current_template=None, current_dataset_id=None, current_job_id=None, current_prompt_optimization_job_id=None, api_url=HttpUrl('https://api.hp.galileocloud.io/'))

In [42]:
comparer = GalileoLocalComparer(
    base_selector=selector,
    finetuned_path=fine_tuned_path,
    prompts=[
        "Explain the importance of sustainable agriculture.",
        "Write a Python function to check for palindromes.",
    ],
    galileo_project_name=GALILEO_EVALUATE_PROJECT_NAME,
    dtype=torch.float16
)

comparer.compare()

INFO:AcceleratedInferenceRunner:🔄 Loading tokenizer and base model from ModelSelector...
INFO:AcceleratedInferenceRunner:✅ Model loaded and ready for inference.
INFO:AcceleratedInferenceRunner:🔄 Loading tokenizer and base model from ModelSelector...
INFO:AcceleratedInferenceRunner:🎯 Applying LoRA fine-tuned weights...
INFO:AcceleratedInferenceRunner:✅ Model loaded and ready for inference.
INFO:AcceleratedInferenceRunner:🔍 Running inference for prompt (truncated): Explain the importance of sustainable agriculture....


⚙️ Running prompt 1/2


INFO:AcceleratedInferenceRunner:✅ Inference complete.
INFO:AcceleratedInferenceRunner:🔍 Running inference for prompt (truncated): Explain the importance of sustainable agriculture....
INFO:AcceleratedInferenceRunner:✅ Inference complete.
INFO:AcceleratedInferenceRunner:🔍 Running inference for prompt (truncated): Write a Python function to check for palindromes....


⚙️ Running prompt 2/2


INFO:AcceleratedInferenceRunner:✅ Inference complete.
INFO:AcceleratedInferenceRunner:🔍 Running inference for prompt (truncated): Write a Python function to check for palindromes....
INFO:AcceleratedInferenceRunner:✅ Inference complete.
INFO:promptquality.utils.logger:Project AIStudio-Fine-Tuning-Evaluate already exists, using it.


Processing chain run...:   0%|          | 0/5 [00:00<?, ?it/s]

Initial job complete, executing scorers asynchronously. Current status:
rag_nli: Computing 🚧
instruction_adherence: Computing 🚧
cost: Done ✅
toxicity: Computing 🚧
pii: Computing 🚧
protect_status: Done ✅
latency: Done ✅
factuality: Computing 🚧
🔭 View your prompt run on the Galileo console at: https://console.hp.galileocloud.io/prompt/chains/ba7eae8e-754b-4db3-8126-65edbb062949/c760d967-a0b1-4019-af4b-ed90abc5d2ff?taskType=12
✅ Finished logging outputs for both models to Galileo.


In [43]:
logger.info('Notebook execution completed.')

2025-06-23 10:28:46 - INFO - Notebook execution completed.
2025-06-23 10:28:46 - INFO - Notebook execution completed.


Built with ❤️ using Z by HP AI Studio.